In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, preprocessing, feature_extraction

%matplotlib inline

plt.style.use('seaborn')

In [8]:
test_users = pd.read_csv("test_users.csv")
train_users = pd.read_csv("train_users_2.csv")
print(train_users.shape)
print(test_users.shape)

(213451, 16)
(62096, 15)


In [10]:
train_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 16 columns):
id                         213451 non-null object
date_account_created       213451 non-null object
timestamp_first_active     213451 non-null int64
date_first_booking         88908 non-null object
gender                     213451 non-null object
age                        125461 non-null float64
signup_method              213451 non-null object
signup_flow                213451 non-null int64
language                   213451 non-null object
affiliate_channel          213451 non-null object
affiliate_provider         213451 non-null object
first_affiliate_tracked    207386 non-null object
signup_app                 213451 non-null object
first_device_type          213451 non-null object
first_browser              213451 non-null object
country_destination        213451 non-null object
dtypes: float64(1), int64(2), object(13)
memory usage: 26.1+ MB


In [3]:
all_users = pd.concat((train_users, test_users), axis = 0, ignore_index = True)
all_users.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
0,direct,direct,NaN,NDF,2010-06-28,NaN,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,20090319043255
1,seo,google,38.0,NDF,2011-05-25,NaN,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,20090523174809
2,direct,direct,56.0,US,2010-09-28,2010-08-02,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,20090609231247
3,direct,direct,42.0,other,2011-12-05,2012-09-08,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,20091031060129
4,direct,direct,41.0,US,2010-09-14,2010-02-18,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,Web,0,basic,20091208061105


In [4]:
sessions = pd.read_csv('sessions.csv')
sessions.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [12]:
sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10567737 entries, 0 to 10567736
Data columns (total 6 columns):
user_id          object
action           object
action_type      object
action_detail    object
device_type      object
secs_elapsed     float64
dtypes: float64(1), object(5)
memory usage: 483.8+ MB


In [5]:
df_sess = sessions.groupby(['user_id']).user_id.count().reset_index(name = 'session_count')
df_sess.head()

,user_id,session_count
0,00023iyk9l,40
1,0010k6l0om,63
2,001wyh0pz8,90
3,0028jgx1x1,31
4,002qnbzfs5,789


In [6]:
secs = sessions.groupby(['user_id']).secs_elapsed.sum().reset_index()
secs.columns = ['user_id', 'secs_elapsed']
secs.describe()

,secs_elapsed
count,1.354830e+05
mean,1.489732e+06
std,1.892923e+06
min,0.000000e+00
25%,2.605980e+05
50%,8.502710e+05
75%,2.000422e+06
max,3.822136e+07


In [7]:
train_users_labels = train_users.loc[:, 'country_destination']
print(train_users_labels.head())

0      NDF
1      NDF
2       US
3    other
4       US
Name: country_destination, dtype: object


In [41]:
train_users = train_users.drop(['date_first_booking'], axis = 1)
test_users = test_users.drop(['date_first_booking'], axis = 1)

In [42]:
#  Split the feature "data_account_created" to "year", "month", "day"
date_acc_created_train = np.vstack(train_users.date_account_created.astype(str).apply(
    lambda x : list(map(int, x.split('-')))).values)

train_users['create_year'] = date_acc_created_train[:, 0]
train_users['create_month'] = date_acc_created_train[:, 1]
train_users['create_day'] = date_acc_created_train[:, 2]
train_users = train_users.drop(['date_account_created'], axis = 1)

date_acc_created_test = np.vstack(test_users.date_account_created.astype(str).apply(
    lambda x : list(map(int, x.split('-')))).values)

test_users['create_year'] = date_acc_created_test[:, 0]
test_users['create_month'] = date_acc_created_test[:, 1]
test_users['create_day'] = date_acc_created_test[:, 2]
test_users = test_users.drop(['date_account_created'], axis = 1)

In [43]:
train_users.loc[train_users.gender == '-unknown-', 'gender'] = -1
train_users.loc[train_users.gender.isnull(), 'gender'] = -1
test_users.loc[test_users.gender == '-unknown-', 'gender'] = -1
test_users.loc[test_users.gender.isnull(), 'gender'] = -1

In [44]:
gender_enc = {'FEMALE' : 0,
             'MALE' : 1,
             'OTHER' : 2,
             -1 : -1}
for data in [train_users, test_users]:
    data.gender = data.gender.apply(lambda x : gender_enc[x])

In [45]:
train_users.loc[train_users.age > 90, 'age'] = np.nan
train_users.loc[train_users.age < 16, 'age'] = np.nan
test_users.loc[test_users.age > 90, 'age'] = np.nan
test_users.loc[test_users.age < 16, 'age'] = np.nan

In [46]:
train_users.loc[train_users.age.isnull(), 'age'] = train_users.age.median()
test_users.loc[test_users.age.isnull(), 'age'] = test_users.age.median()

In [47]:
signup_enc = {'facebook' : 0,
             'google' : 1,
             'basic' : 2,
             'weibo' : 3}
for data in [train_users, test_users]:
    data.signup_method = data.signup_method.apply(lambda x : signup_enc[x])

In [48]:
test_users.loc[test_users.language == '-unknown-', 'language'] = test_users.language.mode()[0]

In [49]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train_users.language = le.fit_transform(train_users.language)
test_users.language = le.fit_transform(test_users.language)

In [50]:
train_users.affiliate_channel = le.fit_transform(train_users.affiliate_channel)
train_users.affiliate_provider = le.fit_transform(train_users.affiliate_provider)
test_users.affiliate_channel = le.fit_transform(test_users.affiliate_channel)
test_users.affiliate_provider = le.fit_transform(test_users.affiliate_provider)

train_users.loc[train_users.first_affiliate_tracked.isnull(), 'first_affiliate_tracked'] = 'untracked'
train_users.first_affiliate_tracked = le.fit_transform(train_users.first_affiliate_tracked)

test_users.loc[test_users.first_affiliate_tracked.isnull(), 'first_affiliate_tracked'] = 'untracked'
test_users.first_affiliate_tracked = le.fit_transform(test_users.first_affiliate_tracked)

In [51]:
train_users.signup_app = le.fit_transform(train_users.signup_app)
train_users.first_device_type = le.fit_transform(train_users.first_device_type)
train_users.first_browser = le.fit_transform(train_users.first_browser)
test_users.signup_app = le.fit_transform(test_users.signup_app)
test_users.first_device_type = le.fit_transform(test_users.first_device_type)
test_users.first_browser = le.fit_transform(test_users.first_browser)

In [52]:
## session
df = sessions.user_id.value_counts()
print(df.shape)
print(df.head())

(135483,)
mxqbh3ykxl    2722
0hjoc5q8nf    2644
mjbl6rrj52    2476
l5lgm3w5pc    2424
wg9413iaux    2362
Name: user_id, dtype: int64


In [53]:
df = df.to_frame()

In [54]:
df = df.rename(columns = {'user_id' : 'session_count'})
df['id'] = df.index
df.head()

,session_count,id
mxqbh3ykxl,2722,mxqbh3ykxl
0hjoc5q8nf,2644,0hjoc5q8nf
mjbl6rrj52,2476,mjbl6rrj52
l5lgm3w5pc,2424,l5lgm3w5pc
wg9413iaux,2362,wg9413iaux


In [55]:
train_users = pd.merge(train_users, df, how = 'left', on = ['id'])
test_users = pd.merge(test_users, df, how = 'left', on = ['id'])

In [56]:
train_users.session_count.fillna(0, inplace = True)
test_users.session_count.fillna(0, inplace = True)

In [57]:
train_users.session_count = train_users.session_count.astype(int)
test_users.session_count = test_users.session_count.astype(int)

## result

In [58]:
label_df = train_users_labels.to_frame()
for data in [label_df]:
    data.country_destination = le.fit_transform(data.country_destination)

label_df.head()

,country_destination
0,7
1,7
2,10
3,11
4,10


In [59]:
train_users = train_users.drop(['id'], axis = 1)
train_users = train_users.drop([ 'timestamp_first_active'], axis = 1)
train_users = train_users.drop(['country_destination'], axis = 1)
train_users.head()

,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,create_year,create_month,create_day,session_count
0,-1,33.0,0,0,5,2,4,6,2,3,8,2010,6,28,0
1,1,38.0,0,0,5,7,8,6,2,3,8,2011,5,25,0
2,0,56.0,2,3,5,2,4,6,2,6,21,2010,9,28,0
3,0,42.0,0,0,5,2,4,6,2,3,17,2011,12,5,0
4,-1,41.0,2,0,5,2,4,6,2,3,8,2010,9,14,0


In [60]:
train_users.shape

(213451, 15)

In [61]:
## 把维度改为一致
test_users1= test_users.drop(['id','timestamp_first_active'],axis=1)
test_users1.head()

,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,create_year,create_month,create_day,session_count
0,0,35.0,0,0,5,1,4,6,1,8,17,2014,7,1,8
1,-1,31.0,2,0,5,1,4,6,1,8,17,2014,7,1,19
2,-1,31.0,2,0,5,1,4,0,2,6,5,2014,7,1,58
3,-1,31.0,2,0,5,1,4,0,2,6,11,2014,7,1,11
4,-1,31.0,2,0,5,1,4,6,2,3,23,2014,7,1,19


In [62]:
test_users.shape

(62096, 17)

In [63]:
train_users = train_users.drop(['create_day'], axis = 1)
test_users1 = test_users1.drop(['create_day'], axis = 1)
test_users1.shape

(62096, 14)

In [64]:
from sklearn import preprocessing
ss = preprocessing.StandardScaler()
train_users_scaled = pd.DataFrame(ss.fit_transform(train_users))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [65]:
train_users_scaled.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.987150,-0.225180,-1.596552,-0.427798,-0.139772,-0.508867,-0.589495,0.811626,-0.018455,-0.928434,-0.754483,-3.222044,-0.006939,-0.345061
1,1.452068,0.339346,-1.596552,-0.427798,-0.139772,2.384576,0.956108,0.811626,-0.018455,-0.928434,-0.754483,-2.156499,-0.315897,-0.345061
2,0.232459,2.371640,0.628333,-0.035009,-0.139772,-0.508867,-0.589495,0.811626,-0.018455,0.644693,0.169757,-3.222044,0.919936,-0.345061
3,0.232459,0.790967,-1.596552,-0.427798,-0.139772,-0.508867,-0.589495,0.811626,-0.018455,-0.928434,-0.114625,-2.156499,1.846811,-0.345061
4,-0.987150,0.678061,0.628333,-0.427798,-0.139772,-0.508867,-0.589495,0.811626,-0.018455,-0.928434,-0.754483,-3.222044,0.919936,-0.345061


# 贝叶斯

In [90]:
# naive bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

In [91]:
gnb = GaussianNB()
gnb.fit(train_users_scaled, label_df.values.ravel())

GaussianNB(priors=None, var_smoothing=1e-09)

In [92]:
test_users_scaled = pd.DataFrame(ss.fit_transform(test_users1))

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [93]:
test_users_scaled.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.392956,0.285999,-1.746586,-0.694308,-0.17566,-0.60435,-0.513851,0.825682,-1.268724,1.071164,1.037029,0.0,-1.182419,-0.645020
1,-0.832054,-0.197869,0.595598,-0.694308,-0.17566,-0.60435,-0.513851,0.825682,-1.268724,1.071164,1.037029,0.0,-1.182419,-0.547080
2,-0.832054,-0.197869,0.595598,-0.694308,-0.17566,-0.60435,-0.513851,-1.538351,-0.023817,0.327464,-0.463274,0.0,-1.182419,-0.199838
3,-0.832054,-0.197869,0.595598,-0.694308,-0.17566,-0.60435,-0.513851,-1.538351,-0.023817,0.327464,0.286878,0.0,-1.182419,-0.618309
4,-0.832054,-0.197869,0.595598,-0.694308,-0.17566,-0.60435,-0.513851,0.825682,-0.023817,-0.788085,1.787181,0.0,-1.182419,-0.547080


In [115]:
country = gnb.predict_proba(test_users_scaled)

In [117]:
ids_test = test_users['id']

ids = []
countries = []

for i in range(len(ids_test)):
    idx = ids_test[i]
    ids += [idx] * 5
    countries += le.inverse_transform(np.argsort(prediction_proba[2])[::-1][:5]).tolist()

In [118]:
submission = pd.DataFrame({
    "id" : ids,
    "country" : countries
})

In [119]:
submission.to_csv('submission_bayes.csv', index = False)

# Linear Discriminant Analysis

In [120]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis()
lda.fit(train_users_scaled, label_df.values.ravel())

LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001)

In [121]:
country_lda = lda.predict_proba(test_users_scaled)

In [122]:
country_lda.shape

(62096, 12)

In [123]:
ids_test = test_users['id']

ids = []
countries = []

for i in range(len(ids_test)):
    idx = ids_test[i]
    ids += [idx] * 5
    countries += le.inverse_transform(np.argsort(prediction_proba[2])[::-1][:5]).tolist()

In [124]:
submission = pd.DataFrame({
    "id" : ids,
    "country" : countries
})
submission.to_csv('submission_lda.csv', index = False)

# GBC

In [104]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(max_depth = 4, n_estimators = 100, random_state = 817)
gb.fit(train_users_scaled, label_df.values.ravel())

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=4,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=817,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [105]:
prediction_proba = gb.predict_proba(test_users_scaled)

In [106]:
ids_test = test_users['id']

ids = []
countries = []

for i in range(len(ids_test)):
    idx = ids_test[i]
    ids += [idx] * 5
    countries += le.inverse_transform(np.argsort(prediction_proba[2])[::-1][:5]).tolist()

In [107]:
submission = pd.DataFrame({
    "id" : ids,
    "country" : countries
})

In [108]:
submission.to_csv('submission_gbc.csv', index = False)

In [111]:
prediction_proba

array([[0.0047076 , 0.00356257, 0.00404965, ..., 0.00075681, 0.23910779,
        0.03300255],
       [0.0019776 , 0.00316663, 0.00269874, ..., 0.00059886, 0.14171044,
        0.02469454],
       [0.00108381, 0.00335359, 0.00245381, ..., 0.00060344, 0.15498117,
        0.03005659],
       ...,
       [0.00270591, 0.00120678, 0.0051907 , ..., 0.00108383, 0.11460251,
        0.01647884],
       [0.00157666, 0.00352772, 0.00145816, ..., 0.00061932, 0.13201836,
        0.02677271],
       [0.00146284, 0.00735705, 0.00245045, ..., 0.00051739, 0.32706686,
        0.03155164]])

In [125]:
ids_test = test_users['id']

ids = []
countries = []

for i in range(len(ids_test)):
    idx = ids_test[i]
    ids += [idx] * 5
    countries += ['NDF','US','other','FR','IT']

submission = pd.DataFrame({
    "id" : ids,
    "country" : countries
})
submission.to_csv('submission_popular.csv', index = False)